In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Query_ERCOT_DB import Query_ERCOT_DB

class Query_CRR(Query_ERCOT_DB):
    
    def __init__(self):
        self.crr_periods = None
        self.sources = {}
        self.sinks = {}
        self.graph_list = []
        self.neighboring_nodes =[]
        Query_ERCOT_DB.c.execute("""SELECT StartDate, EndDate
            FROM crr_ownership GROUP BY StartDate, EndDate""")
        self.crr_periods = list(Query_ERCOT_DB.c.fetchall())[1:]
        Query_ERCOT_DB.c.execute("""SELECT DISTINCT Source
            FROM crr_ownership ORDER BY Source""")
        sources = list(Query_ERCOT_DB.c.fetchall())
        sources = [r[0] for r in sources]
        for idx, src in enumerate(sources):
            self.sources[src] = idx
        Query_ERCOT_DB.c.execute("""SELECT DISTINCT Sink
            FROM crr_ownership ORDER BY Sink""")
        sinks = list(Query_ERCOT_DB.c.fetchall())
        sinks = [r[0] for r in sinks]
        for idx, snk in enumerate(sinks):
            self.sinks[snk] = idx
        
    
    
    def query_origin_node(self,node,option='date_range',start_date='2011-01-01',end_date='2011-12-31' ):
        if option == 'date_range':
            Query_ERCOT_DB.c.execute("""select sink
                FROM crr_ownership 
                WHERE StartDate >= "%s" 
                AND EndDate <= "%s" 
                AND source = "%s"
                GROUP BY source, sink""" % (start_date, end_date, node))
            temp = list(Query_ERCOT_DB.c.fetchall())
            sinks = []
            for row in temp:
                sinks.append(row[0])
            Query_ERCOT_DB.c.execute("""select source
                FROM crr_ownership 
                WHERE StartDate >= "%s" 
                AND EndDate <= "%s" 
                AND sink = "%s"
                GROUP BY source, sink""" % (start_date, end_date, node))
            temp = list(Query_ERCOT_DB.c.fetchall())
            sources = []
            for row in temp:
                sources.append(row[0])
            self.neighboring_nodes = sinks + sources
            
    def query_neighboring_nodes(self):
        for node in self.neighboring_nodes:
            Query_ERCOT_DB.c.execute("""select *
                    FROM crr_ownership 
                    WHERE StartDate >= "%s" 
                    AND EndDate <= "%s" 
                    AND source = "%s"
                    GROUP BY source, sink""" % (start_date, end_date, node))

In [30]:
qcrr = Query_CRR()
qcrr.query_origin_node(node='CHE_LYD')
sorted(qcrr.neighboring_nodes)
    
    

['APD_APD_G1',
 'APD_APD_G1',
 'AZ_ALL',
 'BTE_BTE_G1',
 'BTE_PUN1',
 'BVE_UNIT1',
 'BVE_UNIT2',
 'CAL_CALGT1',
 'CBY4_ALL',
 'CBY4_CT41',
 'CBY4_CT41',
 'CBY_CBY_G1',
 'CBY_CBY_G2',
 'CHEDPW_GT2',
 'CHEDPW_GT2',
 'CHE_LYD2',
 'CHE_LYD2',
 'CHE_PUN1',
 'CHE_PUN1',
 'CHE_PUN2',
 'CHE_PUN2',
 'CTL_PUN3',
 'CVC_CVC_G2',
 'DDPEC_GT1',
 'DDPEC_GT3',
 'DIB_DIB_G1',
 'DOWGEN_PUN1',
 'DUP_DUPV1_G1',
 'EXN_PUN1',
 'GBY_GBY_5',
 'GBY_GT',
 'GBY_GT',
 'HB_HOUSTON',
 'HB_HOUSTON',
 'HB_NORTH',
 'HB_WEST',
 'LHM_CVC_G4',
 'LHM_CVC_G4',
 'LZ_HOUSTON',
 'LZ_HOUSTON',
 'LZ_LCRA',
 'LZ_NORTH',
 'MCSES_UNIT8',
 'MIL_MILG1_2',
 'NUE_NUECESG7',
 'NWF_NWF1',
 'PR_PR_G2',
 'PR_PR_G2',
 'PSA_PSA_G1',
 'PSA_PUN1',
 'PSA_PUN3',
 'PSG_PSG_GT2',
 'PSG_PSG_GT2',
 'PSG_PSG_GT3',
 'PSG_PSG_GT3',
 'PSG_PSG_ST1',
 'PSG_PSG_ST1',
 'SIL_SILAS_5',
 'SL_PUN1',
 'STP_STP_G1',
 'STP_STP_G2',
 'SWEETWND_1',
 'TC_PUN1',
 'TC_PUN1',
 'TC_TCHP1',
 'TC_TCHP1',
 'THW_1_CCU',
 'THW_THWGT5X',
 'THW_THWGT5X',
 'TTWEC_G1',
 'TXCTY_C